In [1]:
import pandas as pd
import numpy as np
import nest_asyncio
import asyncio
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

from utils import read_pcap_file, combine_request_reply, convert_session_log_to_str

nest_asyncio.apply()  # Needed for running the async function in the notebook


pd.set_option('display.max_colwidth', None)  # No truncation of column content

# PCAP files

In [2]:
dir_contents = [i for i in os.listdir("./data") if ".pcap" in i]
dir_contents

['sessionfailureEMS.pcap',
 'EMSSessionNoResp.pcap',
 'EMSSession11.pcap',
 'EMSSecurityAccessFailure.pcap',
 'PreconditionCheckFailA2.pcap',
 'Endless78.pcap',
 'vinfailure_continue.pcap',
 'EMSSession14.pcap',
 'PreconditionCheckNegResp.pcap',
 'EMSSession12.pcap',
 'EMSSession13.pcap',
 'vinfailure_stop.pcap']

# Read PCAP, Convert DataFrame

In [3]:
pcap_file = 'sessionfailureEMS.pcap'

file_path = os.path.join("data", pcap_file)

df = asyncio.run(read_pcap_file(file_path))

df.head(10)

,number,timestamp,source,target,request,sid,error
0,138,2024-09-12 16:50:27,0x0e00,0x1032,True,0x3E,None
1,141,2024-09-12 16:50:27,0x1032,0x0e00,False,0x7E,None
2,144,2024-09-12 16:50:27,0x0e00,0x1032,True,0x3E,None
3,147,2024-09-12 16:50:27,0x1032,0x0e00,False,0x7E,None
4,149,2024-09-12 16:50:27,0x0e00,0x1032,True,0x22,None
5,153,2024-09-12 16:50:27,0x1032,0x0e00,False,0x7F,0x10
6,155,2024-09-12 16:50:27,0x0e00,0x1032,True,0x10,None
7,158,2024-09-12 16:50:27,0x1032,0x0e00,False,0x50,None
8,160,2024-09-12 16:50:27,0x0e00,0x1032,True,0x10,None
9,162,2024-09-12 16:50:27,0x1032,0x0e00,False,0x50,None


# Merge Request and Reply into One Row

In [4]:
reply_request = combine_request_reply(df)
reply_request

,ecu_address,request_sid,reply_sid,error,request_description,reply_description
0,0x1032,0x3E,0x7E,No error,Tester Present Request,Positive Response to Tester Present
1,0x1032,0x3E,0x7E,No error,Tester Present Request,Positive Response to Tester Present
2,0x1032,0x22,0x7F,General Reject,Read Data By Identifier Request,Negative Response
3,0x1032,0x10,0x50,No error,Diagnostic Session Control Request,Positive Response to Diagnostic Session Control
4,0x1032,0x10,0x50,No error,Diagnostic Session Control Request,Positive Response to Diagnostic Session Control
5,0x1032,0x10,0x50,No error,Diagnostic Session Control Request,Positive Response to Diagnostic Session Control
6,0x1032,0x10,0x50,No error,Diagnostic Session Control Request,Positive Response to Diagnostic Session Control
7,0x1032,0x22,0x62,No error,Read Data By Identifier Request,Positive Response to Read Data By Identifier
8,0x1032,0x22,0x62,No error,Read Data By Identifier Request,Positive Response to Read Data By Identifier
9,0x1032,0x22,0x62,No error,Read Data By Identifier Request,Positive Response to Read Data By Identifier


# Convert Session Log to String

In [5]:
session_log = convert_session_log_to_str(reply_request)
print(session_log)

ECU '0x1032': SID 0x3E (Tester Present Request) -> SID 0x7E (Positive Response to Tester Present) // No error
ECU '0x1032': SID 0x3E (Tester Present Request) -> SID 0x7E (Positive Response to Tester Present) // No error
ECU '0x1032': SID 0x22 (Read Data By Identifier Request) -> SID 0x7F (Negative Response) // General Reject
ECU '0x1032': SID 0x10 (Diagnostic Session Control Request) -> SID 0x50 (Positive Response to Diagnostic Session Control) // No error
ECU '0x1032': SID 0x10 (Diagnostic Session Control Request) -> SID 0x50 (Positive Response to Diagnostic Session Control) // No error
ECU '0x1032': SID 0x10 (Diagnostic Session Control Request) -> SID 0x50 (Positive Response to Diagnostic Session Control) // No error
ECU '0x1032': SID 0x10 (Diagnostic Session Control Request) -> SID 0x50 (Positive Response to Diagnostic Session Control) // No error
ECU '0x1032': SID 0x22 (Read Data By Identifier Request) -> SID 0x62 (Positive Response to Read Data By Identifier) // No error
ECU '0x10

# Initialize LLM

In [6]:
# Load environment variables from .env file
load_dotenv()

# Get the OPENAI_API_KEY from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(api_key=openai_api_key, temperature=0, model="gpt-4o")

In [7]:
template = """ The following text file contains a log of messages between a diagnostic tool and one of more of a 
vehicle's Electronic Control Unit (ECU) system. These messages are coded in the Unified Diagnostic Services (UDS) 
protocol. 

Each row represents a request message sent from the diagnostic tool and the ECU's response. The service
ID codes (SIDs) and their descriptions are included in the log. Descriptions of the negative response codes (NRCs), if
any, are included to the right of the '//'.

Using only the information provided, create a concise summary (max. 25 words) of what's happening and any potential 
errors with the system.

Messages: {messages}
"""

prompt = PromptTemplate(
    input_variables=["messages"],
    template=template,
)

chain = prompt | llm

result = chain.invoke({"messages": reply_request.to_string()}).content

print("\nSummary: \n", result)


Summary: 
 The diagnostic tool communicates with ECUs, mostly successfully, but encounters "General Reject" errors during some "Read Data" and "Diagnostic Session" requests.
